In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_citus_db",
                user="citus_bdp_usage_qa",
                host="10.2.10.132",
                password="dNzWtSV3pKTx",
                port=5432
            )
        )
        count_sql = "SELECT category_id,CAST(SUM(est_market_size_download)AS bigint)AS sum_est_market_size_download,CAST(SUM(est_market_size_revenue) AS bigint)AS sum_est_market_size_revenue, count(1) FROM store.store_market_size_fact_v1 WHERE date >='{}' AND date<='{}' group by category_id order by category_id desc ".format(start,end)
        db_data = query(citus_dsn_, count_sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start,end)
    return [Row(category_id=r[0],sum_est_market_size_download=r[1], sum_est_market_size_revenue=r[2], count=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("category_id", StringType(), True),
    StructField("sum_est_market_size_download", LongType(), True),
    StructField("sum_est_market_size_revenue", LongType(), True),
    StructField("count", LongType(), True)
    ])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data")
print("================online=======================")
online_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-31"))
online_df.show()
print("================ADL=========================")
sample_data_df=spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.market-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
spark.sql("SELECT SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue,count(1) FROM sample_data ").show()
test_sample_data_df =spark.sql("select category_id,SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue,count(1) FROM sample_data group by category_id order by category_id desc")
test_sample_data_df.show()
print("column test (except all)")
test_sample_data_df.exceptAll(online_df).createOrReplaceTempView("test_sample_data")
spark.sql("select * from test_sample_data order by category_id desc ").show()

In [0]:



raw_data_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/").where(" granularity='daily' and date >= '2020-08-01' and date <= '2020-08-31'").cache()
raw_data_df.createOrReplaceTempView("raw_data")
# spark.sql("select * from raw_data").show()
# spark.sql("select app_price_type_id, category_id, country_code, device_code, purchase_type_id, granularity, date, count(1) as count_number from raw_data group by app_price_type_id, category_id, country_code, device_code, purchase_type_id, granularity, date order by count_number desc").show()
# spark.sql("select * from raw_data where category_id=400027 and country_code = 'WW' and device_code ='android-all' and  purchase_type_id = 12 and app_price_type_id=0 ").show()

spark.sql("select date, sum(est_market_size_download), sum(est_market_size_revenue), count(1) from (select distinct app_price_type_id, category_id, country_code, device_code, purchase_type_id, granularity, date, est_market_size_download, est_market_size_revenue from raw_data  ) as temp group by date order by date desc").show(50,False)



In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
select date,count(1), sum(est_market_size_download)as download, sum(est_market_size_revenue) as revenue FROM store.store_market_size_fact_v1  WHERE date between '2020-08-01' AND '2020-08-31' group by date order by date desc;
EOF

In [0]:

import psycopg2
import datetime
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_citus_db",
                user="citus_bdp_usage_qa",
                host="10.2.10.132",
                password="dNzWtSV3pKTx",
                port=5432
            )
        )
        count_sql = "SELECT device_code,country_code,category_id,app_price_type_id,purchase_type_id,granularity,est_market_size_download,est_market_size_revenue,count(distinct(app_price_type_id, purchase_type_id,category_id,device_code,country_code,date,granularity)) FROM store.store_market_size_fact_v1 WHERE date >='{}' AND date<='{}'and device_code= 'android-all'  GROUP BY device_code,country_code,category_id,app_price_type_id,purchase_type_id,category_id,granularity,est_market_size_download,est_market_size_revenue".format(start,end)
        db_data = query(citus_dsn_, count_sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start,end)
    return [Row(device_code=r[0], country_code=r[1], category_id=r[2],app_price_type_id=r[3],purchase_type_id=r[4],granularity=r[5],est_market_size_download=r[6],est_market_size_revenue=r[7],count=r[8]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("country_code", StringType(), True),
    StructField("category_id", LongType(), True),
    StructField("app_price_type_id",  LongType(), True),
    StructField("purchase_type_id",  LongType(), True),
    StructField("granularity", StringType(), True),
    StructField("est_market_size_download", LongType(), True),
    StructField("est_market_size_revenue", LongType(), True),
    StructField("count", LongType(), True)
    ])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data")
print("================online=======================")
online_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-01"))
online_df.createOrReplaceTempView("online_data")

online_df.show()
print("================ADL=========================")
# sample_data_df=spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.market-download-revenue.v1/fact/")
# sample_data_df.createOrReplaceTempView("sample_data")
# spark.sql("SELECT device_code,count(1),SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue FROM sample_data where date='2020-08-01' group by device_code").show()

raw_data_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/").where(" granularity='daily' and date >= '2020-08-01' and date <= '2020-08-01'")
raw_data.createOrReplaceTempView("raw_data")
new_raw_data_df = spark.sql("select  device_code,country_code,category_id,app_price_type_id,purchase_type_id,granularity,est_market_size_download,est_market_size_revenue,count(1) from raw_data where date = '2020-08-01' and device_code='android-all' group by device_code,country_code,category_id,app_price_type_id,purchase_type_id,granularity,est_market_size_download,est_market_size_revenue ")
print("new raw data")
new_raw_data_df.show()
print("except")
df2= new_raw_data_df.exceptAll(online_df).cache()
df2.show()
df2.createOrReplaceTempView("df2_data")
# spark.sql("select distinct category_id from df2_data").show()

# spark.sql("Select * from raw_data where date = '2020-08-01' and device_code='android-all' and category_id = 400027 group by_identifier,app_price_type_id,category_id,country_code,est_market_size_download,est_market_size_revenue,purchase_type_id,granularity,date,device_code").show()

In [0]:

spark.sql('''
select * from raw_data WHERE date between '2020-08-01' AND '2020-08-01' and category_id=400027 and country_code='NL' and app_price_type_id = 1 and purchase_type_id =10
''').show()

In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
SELECT * FROM store.store_market_size_fact_v1  WHERE date between '2020-08-01' AND '2020-08-01' and category_id=400027 and country_code='NL' and app_price_type_id = 1 and purchase_type_id =10  limit 3;
EOF